In [1]:
from datasets import load_dataset

In [2]:
data = load_dataset('financial_phrasebank', 'sentences_50agree')

In [3]:
df = data['train']

In [4]:
x = df['sentence']
y = df['label']

In [5]:
from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [7]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline("sentiment-analysis",model=model, tokenizer= tokenizer)

config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

C:\Users\rushi\.conda\envs\Finance\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rushi\.cache\huggingface\hub\models--mrm8488--distilroberta-finetuned-financial-news-sentiment-analysis. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [61]:
def tempo(val):
    print(x_train[val])
    dict = {
        0: 'negative',
        1: 'neutral',
        2: 'positive'
    }
    result = dict[y_train[val]]
    print(result)
    print(classifier.predict(x_train[val]))

In [8]:
news = "LONDON Indian firm Tata Steel announced Friday it will close both blast furnaces at its plant in Port Talbot, Wales, eliminating 2,800 jobs, as part of plans to make its unprofitable U.K. operation"
classifier.predict(news)

[{'label': 'negative', 'score': 0.9928297400474548}]

In [9]:
import requests

url = ('https://newsapi.org/v2/top-headlines?sources=google-news-in&category=business&apiKey=cef60813d9da4655ae7fe2ef2446060a')

url3 = ('https://newsapi.org/v2/top-headlines?country=in&category=business&apiKey=cef60813d9da4655ae7fe2ef2446060a')

def GetNewsOnEverything(query):
    url2 = (f'https://newsapi.org/v2/everything?q={query}&apiKey=cef60813d9da4655ae7fe2ef2446060a')
    response = requests.get(url2)
    dict = {}
    # print(response.json())
    dict = response.json()
    return dict

newsDict = GetNewsOnEverything("Tata Steel")

In [12]:
temp = newsDict['articles'][0]
temp

{'source': {'id': 'bbc-news', 'name': 'BBC News'},
 'author': 'https://www.facebook.com/bbcnews',
 'title': "The Papers: Rwanda plan ‘won’t work’ and steel jobs 'betrayal'",
 'description': 'The papers lead on a variety of stories, from polling on migration, to Tata steel job losses and measles.',
 'url': 'https://www.bbc.co.uk/news/blogs-the-papers-68038593',
 'urlToImage': 'https://ichef.bbci.co.uk/news/1024/branded_news/CCFD/production/_132377425_index-nc.jpg',
 'publishedAt': '2024-01-20T00:13:31Z',
 'content': "Image caption, A variety of stories make the front pages of Saturday's newspapers. The i newspaper leads on a polling about the prime minister's plan to send some asylum seekers to Rwanda for process… [+2724 chars]"}

In [15]:
title = temp['title']
description = temp['description']
content = temp['content']

In [17]:
final_string = title+description+content

In [18]:
classifier.predict(final_string)

[{'label': 'negative', 'score': 0.9740871787071228}]

In [27]:
def get_joined_string(newsDict):
    temp_string = []
    for i in range(len(newsDict['articles'])):
        temp = newsDict['articles'][i]
        title = temp['title']
        description = temp['description']
        content = temp['content']
        try:
            final_string = title+description+content
        except ValueError:
            if type(title) == None:
                title = ""
            if type(description) == None:
                description = ""
            if type(content) == None:
                content = ""
            final_string = title+description+content
        except Exception as e:
            print("An error occurred:", e)
        temp_string.append(final_string)
    return temp_string

In [28]:
temp_string = get_joined_string(newsDict)

An error occurred: can only concatenate str (not "NoneType") to str
An error occurred: can only concatenate str (not "NoneType") to str
An error occurred: can only concatenate str (not "NoneType") to str
An error occurred: can only concatenate str (not "NoneType") to str
An error occurred: can only concatenate str (not "NoneType") to str


In [30]:
score_list = classifier.predict(temp_string)

In [34]:
score_list[0]

{'label': 'negative', 'score': 0.9740871787071228}

In [40]:
def tempo(score_list):
    dict = {
        'negative': -1,
        'neutral': 0,
        'positive': 1
    }
    score = []
    for i in score_list:
        temp = i['label']
        temp = dict[temp]
        score.append(temp)
    total_score = sum(score) 
    return total_score

In [41]:
tempo(score_list)

-9